In [ ]:
!pip install transformers datasets torch

In [ ]:
!git clone https://github.com/yourusername/autocomplete.git
%cd autocomplete

In [ ]:
import sys
sys.path.append('.')

from src.logger import get_logger 